<a href="https://colab.research.google.com/github/primalbioinformatics/drug-informatics/blob/main/Descriptor_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Descriptors

!pip install rdkit
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

df=pd.read_csv("/content/hinger_binder_diversity_set_11k.smi", sep="\t", names=["SMILES","ID"])
df["Canonical_SMILES"]=df["SMILES"].apply(Chem.CanonSmiles)
def calculate_rule_of_five(mol):
  return{
      "MW": Descriptors.MolWt(mol),
      "LogP": Descriptors.MolLogP(mol),
      "HBA": Descriptors.NumHAcceptors(mol),
      "HBD": Descriptors.NumHDonors(mol),
      "RotatableNonds": Descriptors.NumRotatableBonds(mol)
  }

df_descriptors=pd.DataFrame([calculate_rule_of_five(Chem.MolFromSmiles(smiles)) for smiles in df["Canonical_SMILES"]])
df=pd.concat([df, df_descriptors], axis=1)
df.to_csv("output_descriptors.csv", index=False)

In [ ]:
#QED

import os
import sys
from rdkit import Chem
from rdkit.Chem import QED

file_name = sys.argv[1]
sppl = Chem.SDMolSupplier(r"/content/10k-mol.sdf") #we are using sdf file of compounds here

for mol in sppl:
    print( QED.properties( mol ) )

In [ ]:
#GitHub

!pip install rdkit-pypi
!pip install mordred

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
import numpy as np
from mordred import Calculator, descriptors

#INPUT
input_smi= r"/content/hinger_binder_diversity_set_11k.smi"
dataset = pd.read_csv(input_smi, sep="\t", names=["SMILES", "ID"])

#Canonical
def canonical_smiles(smiles):
    mols=[Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

Canon_SMILES = canonical_smiles(dataset.SMILES)
len(Canon_SMILES)
dataset['SMILES']=Canon_SMILES
dataset

#Duplicated smiles
duplicates_smiles = dataset[dataset['SMILES'].duplicated()]['SMILES'].values
len(duplicates_smiles)

dataset[dataset['SMILES'].isin(duplicates_smiles)].sort_values(by=['SMILES'])
dataset_new = dataset.drop_duplicates(subset=['SMILES'])
dataset_new

#Descriptor calculation
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]
    for mol in mols:
        mol=Chem.AddHs(mol)
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

Mol_descriptors,desc_names = RDkit_descriptors(dataset_new['SMILES'])
df_with_200_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)
df_with_200_descriptors

#Mordred descriptor calculation
def All_Mordred_descriptors(data):
    calc = Calculator(descriptors, ignore_3D=False)
    mols = [Chem.MolFromSmiles(smi) for smi in data]

    # pandas df
    df = calc.pandas(mols)
    return df
mordred_descriptors = All_Mordred_descriptors(dataset_new['SMILES'])
mordred_descriptors

RULE OF THREE - 16 FRAGMENTS

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

data = Chem.SDMolSupplier("/content/prep_fragments.sd", removeHs=False)
molecules = [i for i in data]

smiles = [Chem.MolToSmiles(mol) for mol in molecules]

canonical_smiles = [Chem.CanonSmiles(smi) for smi in smiles]

df = pd.DataFrame({"SMILES": canonical_smiles})

def calculate_rule_of_three(mol):
    descriptors = {
        "MW": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "HBA": Descriptors.NumHAcceptors(mol),
        "HBD": Descriptors.NumHDonors(mol)}
    return descriptors

df_descriptors = pd.DataFrame([calculate_rule_of_three(Chem.MolFromSmiles(smi)) for smi in df["SMILES"]])

df = pd.concat([df, df_descriptors], axis=1)

df_rule_of_three = df[(df["MW"] <= 300) & (df["LogP"] <= 3) & (df["HBA"] <= 3) & (df["HBD"] <= 3)]

print(f"Number of molecules fulfilling the Rule of Three: {len(df_rule_of_three)}")

df_rule_of_three.to_csv("rule_of_three_output.csv", index=False)


Number of molecules fulfilling the Rule of Three: 16


In [ ]:
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
selection = ['MW','LogP','HBA','HBD']
sns.pairplot(df[selection])
plt.tight_layout()
plt.savefig('Pairplot_RO3.png',dpi=300)

RULE OF THREE -  10,682 FRAGMENTS

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

data = Chem.SDMolSupplier("/content/10k_prep_fragments.sdf", removeHs=False)
ids = [mol.GetProp("_Name") for mol in data]
molecules = [i for i in data]
smiles = [Chem.MolToSmiles(mol) for mol in molecules]
canonical_smiles = [Chem.CanonSmiles(smi) for smi in smiles]
df = pd.DataFrame({"SMILES": canonical_smiles, "ID": ids})

def calculate_rule_of_three(mol):

    descriptors = {
        "MW": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "HBA": Descriptors.NumHAcceptors(mol),
        "HBD": Descriptors.NumHDonors(mol),
        "Num_Violations": 0,
    }

    # Check each rule (adapted from Response A)
    if descriptors["MW"] > 300:
        descriptors["Num_Violations"] += 1
    if descriptors["LogP"] > 3:
        descriptors["Num_Violations"] += 1
    if descriptors["HBA"] > 3:
        descriptors["Num_Violations"] += 1
    if descriptors["HBD"] > 3:
        descriptors["Num_Violations"] += 1
    return descriptors

# Calculate descriptors and identify valid/invalid molecules
df_descriptors = pd.DataFrame([calculate_rule_of_three(Chem.MolFromSmiles(smi)) for smi in df["SMILES"]])
df = pd.concat([df, df_descriptors], axis=1)

# Define validity criteria (one violation is valid)
df_valid_1 = df[df["Num_Violations"] == 1]
df_valid_0 = df[df["Num_Violations"] == 0]
df_invalid_2 = df[df["Num_Violations"] == 2]
df_invalid_3 = df[df["Num_Violations"] == 3]
df_invalid_4 = df[df["Num_Violations"] == 4]

# Save to CSV files
df_valid_1.to_csv("valid_molecules_1.csv", index=False)
df_valid_0.to_csv("valid_molecules_0.csv", index=False)
df_invalid_2.to_csv("invalid_molecules_2.csv", index=False)
df_invalid_3.to_csv("invalid_molecules_3.csv", index=False)
df_invalid_4.to_csv("invalid_molecules_4.csv", index=False)

print(f"Number of valid molecules with 1 violation: {len(df_valid_1)}")
print(f"Number of valid molecules with 0 violation: {len(df_valid_0)}")
print(f"Number of invalid molecules with 2 violations: {len(df_invalid_2)}")
print(f"Number of invalid molecules with 3 violations: {len(df_invalid_3)}")
print(f"Number of invalid molecules with 4 violations: {len(df_invalid_4)}")


Number of valid molecules with 1 violation: 4767
Number of valid molecules with 0 violation: 5806
Number of invalid molecules with 2 violations: 108
Number of invalid molecules with 3 violations: 1
Number of invalid molecules with 4 violations: 0


In [ ]:
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
#selection = ['MW','LogP','HBA','HBD']
sns.pairplot(df_valid_0)
plt.tight_layout()
plt.savefig('Pairplot_RO3_0_violation.png',dpi=600)

In [ ]:
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
#selection = ['MW','LogP','HBA','HBD']
sns.pairplot(df_valid_1)
plt.tight_layout()
plt.savefig('Pairplot_RO3_1_violation.png',dpi=600)

In [ ]:
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
#selection = ['MW','LogP','HBA','HBD']
sns.pairplot(df_invalid_2)
plt.tight_layout()
plt.savefig('Pairplot_RO3_2_violations.png',dpi=600)

In [ ]:
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
#selection = ['MW','LogP','HBA','HBD']
sns.pairplot(df_invalid_3)
plt.tight_layout()
plt.savefig('Pairplot_RO3_3_violations.png',dpi=600)

3 Fragment Libraries

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np

data = Chem.SDMolSupplier("/content/LC_Diversity_of_General_Fragments.sdf", removeHs=False)
ids = [mol.GetProp("_Name") for mol in data]
molecules = [i for i in data]
smiles = [Chem.MolToSmiles(mol) for mol in molecules]
canonical_smiles = [Chem.CanonSmiles(smi) for smi in smiles]

duplicates = df["SMILES"].duplicated()
duplicates_count = duplicates.sum()
df_duplicates = df[duplicates]
df_duplicates["Count"] = df_duplicates.groupby("SMILES")["ID"].transform("size")
df = pd.concat([df[~duplicates], df_duplicates], ignore_index=True)
print(f"Number of duplicate SMILES: {duplicates_count}")

unique_smiles, unique_indices = np.unique(canonical_smiles, return_index=True)
df = pd.DataFrame({"SMILES": unique_smiles, "ID": [ids[i] for i in unique_indices]})

def calculate_rule_of_three(mol):

    descriptors = {
        "MW": Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "HBA": Descriptors.NumHAcceptors(mol),
        "HBD": Descriptors.NumHDonors(mol),
        "Num_Violations": 0,
    }

    # Check each rule (adapted from Response A)
    if descriptors["MW"] > 300:
        descriptors["Num_Violations"] += 1
    if descriptors["LogP"] > 3:
        descriptors["Num_Violations"] += 1
    if descriptors["HBA"] > 3:
        descriptors["Num_Violations"] += 1
    if descriptors["HBD"] > 3:
        descriptors["Num_Violations"] += 1
    return descriptors

# Calculate descriptors and identify valid/invalid molecules
df_descriptors = pd.DataFrame([calculate_rule_of_three(Chem.MolFromSmiles(smi)) for smi in df["SMILES"]])
df = pd.concat([df, df_descriptors], axis=1)

# Define validity criteria (one violation is valid)
df_valid_1 = df[df["Num_Violations"] == 1]
df_valid_0 = df[df["Num_Violations"] == 0]
df_invalid_2 = df[df["Num_Violations"] == 2]
df_invalid_3 = df[df["Num_Violations"] == 3]
df_invalid_4 = df[df["Num_Violations"] == 4]

# Save to CSV files
df_valid_1.to_csv("valid_molecules_1.csv", index=False)
df_valid_0.to_csv("valid_molecules_0.csv", index=False)
df_invalid_2.to_csv("invalid_molecules_2.csv", index=False)
df_invalid_3.to_csv("invalid_molecules_3.csv", index=False)
df_invalid_4.to_csv("invalid_molecules_4.csv", index=False)

print(f"Number of valid molecules with 1 violation: {len(df_valid_1)}")
print(f"Number of valid molecules with 0 violation: {len(df_valid_0)}")
print(f"Number of invalid molecules with 2 violations: {len(df_invalid_2)}")
print(f"Number of invalid molecules with 3 violations: {len(df_invalid_3)}")
print(f"Number of invalid molecules with 4 violations: {len(df_invalid_4)}")

<ipython-input-7-1370505cf1d5>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicates["Count"] = df_duplicates.groupby("SMILES")["ID"].transform("size")


Number of duplicate SMILES: 3200
Number of valid molecules with 1 violation: 39230
Number of valid molecules with 0 violation: 20103
Number of invalid molecules with 2 violations: 2235
Number of invalid molecules with 3 violations: 101
Number of invalid molecules with 4 violations: 4
